In [1]:
import pandas as pd

In [2]:
!pip install gpboost mne rdkit aeon neurokit2 heartpy pandarallel

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.1/693.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.7 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=bd65a165d3607086b8f7dd552e540d7dd7354a85c0dba3e2e2889a0bc0dfc051
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.11.12


In [4]:
!rm -rf PyHealth
!git clone https://github.com/sunlabuiuc/PyHealth.git

import sys
sys.path.append("./PyHealth")

Cloning into 'PyHealth'...
remote: Enumerating objects: 8749, done.
remote: Counting objects: 100% (1515/1515), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 8749 (delta 1218), reused 1020 (delta 1001), pack-reused 7234 (from 2)
Receiving objects: 100% (8749/8749), 123.53 MiB | 22.83 MiB/s, done.
Resolving deltas: 100% (5689/5689), done.


In [5]:
%cd "/content/drive/My Drive/CS598DLH/DREAMT_FE"

/content/drive/My Drive/CS598DLH/DREAMT_FE


# Generate an aggregated combined domain features for all patients




In [24]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

# Import your original functions
from datasets import *
from feature_engineering import *

root = "dataset_sample/E4_aggregate_subsample"
info_dir = "dataset_sample/participant_info.csv"
feature_df_dir = "dataset_sample/features_df/"
threshold = 0.2

# Get subject IDs
all_sids = pd.read_csv(info_dir).SID.to_list()
print(all_sids)

for sid in tqdm(all_sids, desc="Processing raw data"):
    domain_features_df = pd.read_csv(os.path.join(feature_df_dir, f"{sid}_domain_features_df.csv"))

# Run the original data_preparation pipeline on processed files
quality_df_dir = "results/quality_scores_per_subject.csv"
clean_df, new_features, good_quality_sids = data_preparation(
    threshold=threshold,
    quality_df_dir=quality_df_dir,
    features_dir=feature_df_dir,
    info_dir=info_dir
)
SW_df, final_features = split_data(clean_df, good_quality_sids, new_features)
print(SW_df.columns.to_list())

SW_df = SW_df.rename(columns={
    'HRV_CD': 'HRV_CD_UPPER',
    'HRV_Cd': 'HRV_Cd_MIXED',
    'rolling_var_HRV_CD': 'rolling_var_HRV_CD_UPPER',
    'rolling_var_HRV_Cd': 'rolling_var_HRV_Cd_MIXED',
    'gaussian_HRV_Cd_1st_derivative': 'gaussian_HRV_Cd_1st_derivative_MIXED',
    'gaussian_HRV_CD_1st_derivative': 'gaussian_HRV_CD_1st_derivative_UPPER',
    'raw_HRV_CD_1st_derivative': 'raw_HRV_CD_1st_derivative_UPPER',
    'raw_HRV_Cd_1st_derivative': 'raw_HRV_Cd_1st_derivative_MIXED'
})

SW_df['timestamp_start'] = pd.to_datetime(SW_df['timestamp_start'], unit='s')
SW_df.to_csv('dataset_sample/all_patients_domain_features.csv', index=False)


['S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S029', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S040', 'S042', 'S043', 'S044', 'S045', 'S046', 'S047', 'S048', 'S049', 'S050', 'S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057', 'S058', 'S059', 'S061', 'S062', 'S063', 'S064', 'S065', 'S066', 'S067', 'S068', 'S069', 'S070', 'S071', 'S072', 'S073', 'S074', 'S075', 'S076', 'S077', 'S078', 'S079', 'S080', 'S081', 'S082', 'S083', 'S084', 'S085', 'S086', 'S087', 'S088', 'S089', 'S090', 'S091', 'S092', 'S093', 'S094', 'S095', 'S096', 'S097', 'S098', 'S099', 'S100', 'S101', 'S102', 'S103']


Processing raw data: 100%|██████████| 100/100 [00:06<00:00, 15.47it/s]


['PPG_Rate_Mean', 'HRV_SDNN', 'HRV_MadNN', 'HRV_SDRMSSD', 'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_MinNN', 'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'HRV_HF', 'HRV_VHF', 'HRV_HFn', 'HRV_SD1SD2', 'HRV_S', 'HRV_CVI', 'HRV_CSI_Modified', 'HRV_PIP', 'HRV_IALS', 'HRV_PSS', 'HRV_PAS', 'HRV_GI', 'HRV_PI', 'HRV_C1d', 'HRV_C2d', 'HRV_Cd', 'HRV_DFA_alpha1', 'HRV_MFDFA_alpha1_Width', 'HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Mean', 'HRV_MFDFA_alpha1_Max', 'HRV_MFDFA_alpha1_Delta', 'HRV_MFDFA_alpha1_Asymmetry', 'HRV_MFDFA_alpha1_Fluctuation', 'HRV_ApEn', 'HRV_ShanEn', 'HRV_FuzzyEn', 'HRV_CD', 'HRV_HFD', 'HRV_KFD', 'HRV_LZC', 'total_power', 'normalized_power', 'HF_frequency_power', 'LF_frequency_peak', 'HF_frequency_peak', 'HR_mean', 'HR_max', 'HR_min', 'BVP_mean', 'BVP_median', 'BVP_max', 'BVP_std', 'ACC_X_trimmed_mean', 'ACC_X_trimmed_max', 'ACC_Y_trimmed_mean', 'ACC_Z_trimmed_mean', 'ACC_X_MAD_trimmed_IQR', 'ACC_INDEX', 'TEMP_mean', 'TEMP_std', 'mean_SCR_Height', 'mean_SCR_RiseTime', 'max_SCR_

# Create a Pyhealth Classification Task

In [12]:
from typing import Any, Dict, List

from pyhealth.tasks.base_task import BaseTask
import numpy as np


class DREAMTE4SleepingStageClassification(BaseTask):
    """A task for classifying sleep stages from physiological signals.

    This task processes wearable device (E4) data to classify sleep stages (Sleep pr Wake)
    using extracted physiological features. It expects sequential physiological measurements
    and returns corresponding sleep stage labels.

    Attributes:
        task_name (str): The name of the task, set to "DREAMTClassification".
        input_schema (Dict[str, str]): The input schema specifying the required
            input format. Contains:
            - "features": "ndarray" (physiological features as numpy array)
        output_schema (Dict[str, str]): The output schema specifying the output
            format. Contains:
            - "label": "binary" (sleep stage classification label)

    Note:
        - Input features include HRV, movement, and other physiological metrics
        - Sleep stages are mapped as: "P": 1, "N": 0, "R": 0, "W": 1, "Missing": np.nan
        - Each sample represents a 30-second epoch of sleep data
    """

    task_name: str = "DREAMTE4SleepingStageClassification"
    input_schema: Dict[str, str] = {"features": "text"}
    output_schema: Dict[str, str] = {"label": "binary"}

    def __call__(self, patient: Any) -> List[Dict[str, Any]]:
        """Process a patient's physiological data to classify sleep stages.

        Args:
            patient: A patient object containing physiological recordings and
                sleep stage annotations. Expected to have events of type
                "dreams_features" containing:
                - sid: Subject ID
                - Sleep_Stage: Annotated sleep stage
                - Various physiological features (HRV metrics, movement, etc.)
                - Respiratory event markers
                - Demographic information

        Returns:
            List[Dict[str, Any]]: A list of dictionaries, each containing:
                - "patient_id": Unique patient identifier
                - "features": Physiological features as numpy array (shape 1×N)
                - "label": Sleep stage classification 0 or 1

        """
        samples = []
        cols_to_remove = [
            "sid", "Sleep_Stage", "Central_Apnea", "Obstructive_Apnea",
            "Multiple_Events", "Hypopnea", 'AHI_Severity', 'Obesity',
            'BMI', "circadian_decay", "circadian_linear", "circadian_cosine",
            "timestamp_start"
        ]

        records = patient.get_events(event_type="dreamt_features")
        for record in records:
            attr = record.attr_dict
            features = [float(v) for k, v in attr.items() if k not in cols_to_remove]

            sample = {
                "patient_id": patient.patient_id[0],
                "features": np.array(features).reshape(1, -1),
                "label": int(float(attr["Sleep_Stage"])),
            }
            samples.append(sample)

        return samples



# Create a Pyhealth Dataset

In [13]:
import logging
import os
from pathlib import Path
from typing import Optional

import pandas as pd

# from pyhealth.tasks import DREAMTE4SleepingStageClassification
from pyhealth.datasets import BaseDataset, SampleDataset
# Import your original functions
from datasets import *
from feature_engineering import *

logger = logging.getLogger(__name__)


class DREAMTE4Dataset(BaseDataset):
    """Base dataset for the DREAMT sleep study dataset.

    The DREAMT dataset contains polysomnography recordings with sleep stage annotations
    and respiratory event information. This dataset is designed for sleep stage
    classification tasks using physiological signals.

    Dataset is available at:
    https://physionet.org/content/dreamt/1.0.0/

    Data Description:
    ----------------
    - Contains polysomnography recordings from sleep studies
    - Includes sleep stage annotations (0-4 corresponding to W, N1, N2, N3, REM)
    - Provides respiratory event information (apneas, hypopneas)
    - Contains derived physiological features for each 30-second epoch
    - Includes demographic and clinical information (BMI, AHI severity)

    Paper:
    Addressing Wearable Sleep Tracking Inequity: A New Dataset and Novel Methods for a Population with Sleep Disorders
    https://raw.githubusercontent.com/mlresearch/v248/main/assets/wang24a/wang24a.pdf

    References:
    ----------
    [1] https://onlinelibrary.wiley.com/doi/abs/10.1002/0471751723.ch1.
    [2] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3817449/.
    [3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4400203/.
    [4] https://my.clevelandclinic.org/health/articles/11429-common-sleep-disorders.
    [5] https://www.nature.com/articles/s41746-020-0244-4.
    [6] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5624990/.
    [7] https://github.com/armiro/COVID-CXNet
    [8] https://www.kaggle.com/c/rsna-pneumonia-detection-challenge/data
    [9] https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

    Args:
        root: Root directory containing the dataset files.
        tables: List of tables to load (default: ["dreams_features"]).
        dataset_name: Optional name of the dataset. Defaults to "dreamt".
        config_path: Optional path to the configuration file. If not provided,
            uses "dreamt.yaml" as default.

    Attributes:
        root: Root directory of the dataset.
        tables: List of loaded tables.
        dataset_name: Name of the dataset.
        config_path: Path to the configuration file.
        patients: Dictionary of processed patient records containing:
            - patient_id: Unique patient identifier
            - records: List of sleep epochs with:
                * record_id: Unique epoch identifier
                * features: Physiological features (numpy array)
                * label: Sleep stage (0-4)

    Examples:
        >>> from pyhealth.datasets import DreamtDataset
        >>> dataset = DreamtDataset(
        ...     root="/path/to/dreamt_data"
        ... )
        >>> dataset.stat()
        >>> samples = dataset.set_task()
        >>> print(samples[0])

    Note:
        The dataset requires pre-processed feature files in CSV format containing:
        - sid: Patient/study identifier
        - Sleep_Stage: Annotated sleep stage (0-4)
        - Various physiological features
        - Respiratory event markers
        - Demographic information
    """

    def __init__(
        self,
        root: str,
        dataset_name: Optional[str] = None,
        config_path: Optional[str] = None,
    ) -> None:
        if config_path is None:
            logger.info("No config path provided, using default config")
            config_path = "dreamt.yaml"
        if not os.path.exists(os.path.join(root, "all_patients_domain_features.csv")):
          self.prepare_metadata(root)
        default_tables = ["dreamt_features"]
        super().__init__(
            root=root,
            tables=default_tables,
            dataset_name=dataset_name or "dreamt",
            config_path=config_path,
        )
        # self.patients = self.process()
        return


    def prepare_metadata(self, root: str) -> None:
        """Prepare metadata for the DREAMT dataset by:
        1. Loading raw feature files
        2. Applying quality threshold 0.2
        3. Processing and renaming features
        4. Saving combined metadata to CSV

        Args:
            root: Root directory containing the dataset files.
        """
        # Set paths
        info_dir = os.path.join(root, "participant_info.csv")
        feature_df_dir = os.path.join(root, "features_df/")
        quality_df_dir = os.path.join(root, "quality_scores_per_subject.csv")
        data_folder = os.path.join(root,"E4_aggregate_subsample")

        # Get subject IDs
        all_sids = pd.read_csv(info_dir).SID.to_list()
        logger.info(f"Found {len(all_sids)} subjects in participant info")

        # Extract and aggregated domain features for E4 data
        for sid in all_sids:
            print(sid)
            try:
                # print(data_folder)
                extract_domain_features(
                    sid, data_folder=data_folder, segment_seconds=30, save_folder_dir= feature_df_dir
                )
            except:
                print("ERROR")

        # Verify feature files exist
        for sid in all_sids:
            feature_file = os.path.join(feature_df_dir, f"{sid}_domain_features_df.csv")
            if not os.path.exists(feature_file):
                logger.warning(f"Missing feature file for subject {sid}")

        # Calculate quality score for each participant
        self.calculate_qaulity_score(root, feature_df_dir)

        # Run data preparation pipeline
        logger.info("Processing raw data with quality threshold 0.2")
        clean_df, new_features, good_quality_sids = data_preparation(
            threshold=0.2,
            quality_df_dir=quality_df_dir,
            features_dir=feature_df_dir,
            info_dir=info_dir
        )

        # Split and process data
        SW_df, final_features = split_data(clean_df, good_quality_sids, new_features)
        logger.info(f"Processed data with {len(SW_df)} records")

        # Rename columns for consistency
        SW_df = SW_df.rename(columns={
            'HRV_CD': 'HRV_CD_UPPER',
            'HRV_Cd': 'HRV_Cd_MIXED',
            'rolling_var_HRV_CD': 'rolling_var_HRV_CD_UPPER',
            'rolling_var_HRV_Cd': 'rolling_var_HRV_Cd_MIXED',
            'gaussian_HRV_Cd_1st_derivative': 'gaussian_HRV_Cd_1st_derivative_MIXED',
            'gaussian_HRV_CD_1st_derivative': 'gaussian_HRV_CD_1st_derivative_UPPER',
            'raw_HRV_CD_1st_derivative': 'raw_HRV_CD_1st_derivative_UPPER',
            'raw_HRV_Cd_1st_derivative': 'raw_HRV_Cd_1st_derivative_MIXED'
        })

        # Convert and save timestamp
        SW_df['timestamp_start'] = pd.to_datetime(SW_df['timestamp_start'], unit='s')

        # Save final metadata
        output_path = os.path.join(root, "all_patients_domain_features.csv")
        SW_df.to_csv(output_path, index=False)
        logger.info(f"Saved processed metadata to {output_path}")
        return

    def calculate_qaulity_score(self, root, feature_df_dir):
        """ Code used to calculate quality score of each participant
        """
        files = os.listdir(feature_df_dir)
        files.sort()

        sids = []
        total_segments = []
        num_excludes = []
        percentages = []

        for file in files:
            sid = file.split('_')[0]
            file_path = str(feature_df_dir + file)
            df = pd.read_csv(file_path)
            segment_len = len(df)
            segment_exclude = np.sum(df.artifact)
            percentage = segment_exclude / segment_len

            sids.append(sid)
            total_segments.append(segment_len)
            num_excludes.append(segment_exclude)
            percentages.append(percentage)

        qs = pd.DataFrame({'sid': sids,
                        'total_segments': total_segments,
                        'num_excludes': num_excludes,
                        'percentage_excludes': percentages})
        output_path = os.path.join(root, "quality_scores_per_subject.csv")
        qs.to_csv(output_path, index=False)
        return


    @property
    def default_task(self) -> DREAMTE4SleepingStageClassification:
        """Returns the default task for this dataset.

        Returns:
             DREAMTE4SleepingStageClassification: The default classification task.
        """
        return DREAMTE4SleepingStageClassification()



root = "dataset_sample"
dreamt_dataset = DREAMTE4Dataset(
    root=root
)


Initializing dreamt dataset from dataset_sample


INFO:pyhealth.datasets.base_dataset:Initializing dreamt dataset from dataset_sample


Scanning table: dreamt_features from dataset_sample/all_patients_domain_features.csv


INFO:pyhealth.datasets.base_dataset:Scanning table: dreamt_features from dataset_sample/all_patients_domain_features.csv


In [14]:
len(dreamt_dataset.get_patient('S003').get_events())

1056

## Prepare data samples for LightGBM model

In [15]:
root = "dataset_sample"
dreamt_dataset = DREAMTE4Dataset(
    root=root
)

task = DREAMTE4SleepingStageClassification()
dreamt_samples = dreamt_dataset.set_task(task)
print(dreamt_samples.input_schema)
print(dreamt_samples.output_schema)
print(len(dreamt_samples))

Initializing dreamt dataset from dataset_sample


INFO:pyhealth.datasets.base_dataset:Initializing dreamt dataset from dataset_sample


Scanning table: dreamt_features from dataset_sample/all_patients_domain_features.csv


INFO:pyhealth.datasets.base_dataset:Scanning table: dreamt_features from dataset_sample/all_patients_domain_features.csv


Setting task for dreamt base dataset...


INFO:pyhealth.datasets.base_dataset:Setting task for dreamt base dataset...
Generating samples for DREAMTE4SleepingStageClassification: 80it [01:02,  1.28it/s]
Processing samples: 100%|██████████| 82671/82671 [00:01<00:00, 77729.96it/s]

{'features': 'text'}
{'label': 'binary'}
82671


In [16]:
list(dreamt_samples.input_schema.keys())

['features']

In [17]:
dreamt_samples.input_schema

{'features': 'text'}

In [18]:
print(dreamt_samples[0])

{'patient_id': 'S004', 'features': array([[ 7.37663552e+01,  2.52376767e+02,  2.31656250e+02,
         7.17673668e-01,  5.93750000e+02,  1.07812500e+03,
         7.71428571e+01,  5.31250000e+02,  1.26562500e+03,
         8.75000000e+00,  6.25000000e+01,  1.24288029e-01,
         2.57271689e-02,  8.28502917e-01,  9.83612822e-01,
         2.03459836e+05,  6.01544880e+00,  1.04348961e+03,
         6.00000000e-01,  6.47058824e-01,  8.18181818e-01,
         2.85714286e-01,  4.94827586e+01,  5.29411765e+01,
         4.71199628e-01,  3.83753585e-01,  4.26755583e-01,
         5.47959782e-01,  3.17686164e+00,  1.17029068e+00,
         1.87763316e+00, -1.59906166e+00, -2.01792507e+00,
        -2.77345519e-01,  1.23055425e-02,  3.84840678e-01,
         4.22900373e+00,  1.65118030e+00,  1.16986539e+00,
         1.99295559e+00,  4.90671531e+00,  1.61206038e+00,
         9.59289062e+02,  3.58459200e+02,  1.19228147e+02,
         6.66666667e-02,  2.00000000e-01,  8.99909220e+01,
         1.39477866e+

In [19]:
print(dreamt_samples[0]['features'].shape)

(1, 358)


In [20]:
print(len(dreamt_samples))

82671


# Create a Pyhealth LightGBM Model

In [21]:
from pyhealth.models import BaseModel
import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
import pickle
from typing import List, Dict
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc, precision_recall_curve, roc_auc_score, cohen_kappa_score

class LightGBMClassifier(BaseModel):
    """LightGBM classifier for PyHealth that handles epoch_path storage.

    Args:
        dataset: PyHealth dataset (with epoch_path in samples)
        feature_keys: Should be ["signal"] for your case
        label_key: Key containing labels ("label")
        mode: "binary" or "multiclass"
        **kwargs: LightGBM parameters
    """
    def __init__(
        self,
        dataset,
        feature_keys: List[str],
        label_key: str,
        mode: str,
        train_test_split: List[float] = None,
        **kwargs
    ):
        self.feature_keys = feature_keys
        self.label_key = label_key
        self.train_test_split = train_test_split
        self.train_samples, self.val_samples, self.test_samples = split_by_visit(dataset, train_test_split)


        super().__init__(
            dataset=dataset,
        )

        self.params = {
            "objective": "binary" if mode == "binary" else "multiclass",
            "verbose": -1,
            **kwargs
        }
        self.model = None

    def fit(self):
        """Train the model using PyHealth data samples."""
        X_train, y_train = self._samples_to_arrays(self.train_samples)
        print(X_train.shape)
        print(y_train.shape)

        eval_set = None
        if self.val_samples:
            X_val, y_val = self._samples_to_arrays(self.val_samples)
            eval_set = [(X_val, y_val)]

        self.model = lgb.LGBMClassifier(**self.params)
        self.model.fit(X_train, y_train, eval_set=eval_set)

    def predict(self):
        """Make predictions using a data samples."""
        X, _ = self._samples_to_arrays(self.test_samples)
        return self.model.predict(X)

    def predict_proba(self):
        """Get prediction probabilities."""
        X, _ = self._samples_to_arrays(self.test_samples)
        return self.model.predict_proba(X)

    def evaluate(self, model_name="LightGBM"):
        """Complete evaluation with metrics and visualization."""
        X_test, y_test = self._samples_to_arrays(self.test_samples)
        y_pred = self.predict()
        y_pred_proba = self.predict_proba()

        # Metrics
        results = []
        accuracy = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        precision = precision_score(
            y_test,
            y_pred,
            labels=[1],
        )
        recall = recall_score(
            y_test,
            y_pred,
            labels=[1],
        )  # Recall is the same as sensitivity
        f1 = f1_score(
            y_test,
            y_pred,
            labels=[1],
        )
        auroc = roc_auc_score(y_test, y_pred_proba[:, 1])
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba[:,1])
        precision_recall_auc = auc(recalls, precisions)
        y_pred_kappa = np.argmax(y_pred_proba, axis=1)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)

        report = classification_report(y_test, y_pred)
        print(report)
        results.append(
            {
                "Model": model_name,
                "Precision": precision,
                "Recall": recall,
                "F1 Score": f1,
                "AUROC": auroc,
                'AUPRC': precision_recall_auc,
                "Accuracy": accuracy,
                "Cohen's Kappa": cohen_kappa,
            }
        )

        return results


    def _samples_to_arrays(self, samples):
        """Convert samples to numpy arrays, concatenating multiple feature keys."""
        X = []
        y = []

        for sample in samples:
            features_list = []
            for feature_key in self.feature_keys:
                feature = sample[feature_key]
                if isinstance(feature, torch.Tensor):
                    feature = feature.numpy()
                # Flatten if needed
                if hasattr(feature, 'flatten'):
                    feature = feature.flatten()
                features_list.append(feature)

            combined_features = np.concatenate(features_list)
            X.append(combined_features)

            label = sample[self.label_key]
            if isinstance(label, torch.Tensor):
                label = label.item()
            y.append(sample[self.label_key])

        return np.array(X), np.array(y)

## Train a LightGBm model for DREAMT E4 in pyhealth

In [23]:
from pyhealth.datasets import split_by_visit

# Initialize
lgb_model = LightGBMClassifier(
    dataset=dreamt_samples,
    train_test_split=[0.7, 0.1, 0.2],
    feature_keys=["features"],
    label_key="label",
    mode="binary",  # or "multiclass"
    learning_rate=0.01,
    n_estimators=300
)

# Train
lgb_model.fit()

# Evaluate
report = lgb_model.evaluate()
report

(57869, 358)
(57869, 1)
              precision    recall  f1-score   support

         0.0       0.90      0.80      0.85      6916
         1.0       0.87      0.94      0.90      9619

    accuracy                           0.88     16535
   macro avg       0.88      0.87      0.87     16535
weighted avg       0.88      0.88      0.88     16535



[{'Model': 'LightGBM',
  'Precision': 0.867013587742122,
  'Recall': 0.9353363135461067,
  'F1 Score': 0.8998799759951991,
  'AUROC': np.float64(0.9452658582327932),
  'AUPRC': np.float64(0.954917811491508),
  'Accuracy': 0.8789234956153613,
  "Cohen's Kappa": np.float64(0.7473069965943959)}]

# Train a RNN model for DREAMT E4 with pyhealth (converting features in *Tensor*)

In [25]:
from typing import Any
import numpy as np
import torch
from pyhealth.processors import register_processor
from pyhealth.processors.base_processor import FeatureProcessor

@register_processor("NumericToTensor")
class NumericProcessor(FeatureProcessor):
    """Processor for numerical feature arrays that converts inputs to PyTorch tensors.

    This processor:
    - Accepts various numerical inputs (lists, numpy arrays, etc.)
    - Converts them to float32 tensors
    - Preserves numerical values exactly
    - Handles type conversion automatically

    Typical use cases:
    - Processing continuous-valued features
    - Converting pre-computed features for neural networks
    - Preparing numerical data for PyTorch models
    """

    def __init__(self):
        super().__init__()

    def process(self, value: Any) -> torch.Tensor:
        """Converts numerical arrays to float32 tensors.

        Args:
            value: Input numerical data (list, numpy array, torch.Tensor, or similar)

        Returns:
            torch.FloatTensor: Converted tensor with dtype=float32

        Raises:
            ValueError: If input cannot be converted to numerical tensor
        """
        # If already a tensor, just ensure correct dtype
        if isinstance(value, torch.Tensor):
            return value.float()

        try:
            # Convert to numpy array if not already
            if not isinstance(value, np.ndarray):
                value = np.array(value, dtype=np.float32)

            # Ensure numerical dtype
            if not np.issubdtype(value.dtype, np.number):
                value = value.astype(np.float32)

            return torch.from_numpy(value).float()
        except Exception as e:
            raise ValueError(f"Cannot convert input to tensor: {value}") from e

    def size(self) -> int:
        """Returns feature dimension (1 for scalar processors).

        Note:
            This processor doesn't maintain a vocabulary size since it handles
            continuous numerical features. The return value of 1 indicates
            it processes single features (though they may be multi-dimensional).
        """
        return 1

    def __repr__(self) -> str:
        return "TensorProcessor()"

In [26]:
from typing import Any, Dict, List

from pyhealth.tasks.base_task import BaseTask
import numpy as np


class DREAMTE4TensorClassification(BaseTask):

    task_name: str = "DREAMTE4SleepingStageClassification"
    input_schema: Dict[str, str] = {"features": "NumericToTensor"}
    output_schema: Dict[str, str] = {"label": "binary"}

    def __call__(self, patient: Any) -> List[Dict[str, Any]]:
        samples = []
        cols_to_remove = [
            "sid", "Sleep_Stage", "Central_Apnea", "Obstructive_Apnea",
            "Multiple_Events", "Hypopnea", 'AHI_Severity', 'Obesity',
            'BMI', "circadian_decay", "circadian_linear", "circadian_cosine",
            "timestamp_start"
        ]

        records = patient.get_events(event_type="dreamt_features")
        for record in records:
            attr = record.attr_dict
            features = [float(v) for k, v in attr.items() if k not in cols_to_remove]

            sample = {
                "patient_id": patient.patient_id[0],
                "features": np.array(features).reshape(1, -1),
                "label": int(float(attr["Sleep_Stage"])),
            }
            samples.append(sample)

        return samples



In [27]:
RNNtask = DREAMTE4TensorClassification()
dreamt_samples_RNN = dreamt_dataset.set_task(RNNtask)
print(dreamt_samples_RNN.input_schema)
print(dreamt_samples_RNN.output_schema)
print(len(dreamt_samples_RNN))

Setting task for dreamt base dataset...


INFO:pyhealth.datasets.base_dataset:Setting task for dreamt base dataset...
Generating samples for DREAMTE4SleepingStageClassification: 80it [01:01,  1.29it/s]
Processing samples: 100%|██████████| 82671/82671 [00:02<00:00, 35186.67it/s]


{'features': 'NumericToTensor'}
{'label': 'binary'}
82671


In [29]:
from pyhealth.datasets import split_by_visit, get_dataloader
from pyhealth.trainer import Trainer
from pyhealth.models import SparcNet, RNN


# Data split
train_samples, val_samples, test_samples = split_by_visit(dreamt_samples_RNN, [0.7, 0.1, 0.2])

# Create dataloaders
train_loader = get_dataloader(train_samples, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_samples, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_samples, batch_size=32, shuffle=False)


model = RNN(
    dataset=dreamt_samples,
    embedding_dim=358,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3
)

trainer = Trainer(
    model=model,
    device="cpu",
    metrics=["roc_auc", "pr_auc", "f1", "accuracy"],
)

trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=10,
    optimizer_class=torch.optim.Adam,  # Optimizer choice
    optimizer_params={"lr": 0.02, "weight_decay": 1e-5},  # Optimizer parameters
    monitor="pr_auc",  # Monitor AUC-ROC for best model selection
    monitor_criterion="max",  # Maximize AUC-ROC during training
    load_best_model_at_last=True,  # Automatically load the best-performing model
)


RNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (rnn): ModuleDict(
    (features): RNNLayer(
      (dropout_layer): Dropout(p=0.3, inplace=False)
      (rnn): GRU(358, 256, num_layers=2, batch_first=True, dropout=0.3)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:RNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (rnn): ModuleDict(
    (features): RNNLayer(
      (dropout_layer): Dropout(p=0.3, inplace=False)
      (rnn): GRU(358, 256, num_layers=2, batch_first=True, dropout=0.3)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['roc_auc', 'pr_auc', 'f1', 'accuracy']


INFO:pyhealth.trainer:Metrics: ['roc_auc', 'pr_auc', 'f1', 'accuracy']


Device: cpu


...

INFO:pyhealth.trainer:accuracy: 0.4234


loss: 0.7615


INFO:pyhealth.trainer:loss: 0.7615


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [30]:
dreamt_results = trainer.evaluate(test_loader)

# Print evaluation metrics
print("dreamt Test Set Evaluation Metrics:", dreamt_results)

Evaluation: 100%|██████████| 517/517 [00:02<00:00, 187.66it/s]

dreamt Test Set Evaluation Metrics: {'roc_auc': np.float64(0.6013204957461298), 'pr_auc': np.float64(0.6598122433682485), 'f1': 0.009476720230737536, 'accuracy': 0.41844572119745993, 'loss': 0.7985152897336727}
